# Assignment 1: Data Prep & Analysis - kelvin.ahiakpor

### Imports

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import requests
import re

### Web Scraping

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

In [ ]:
table = soup.find('table')

#print(table)
#soup.find_all('th')[:7]

##### Getting the column headings from soup as a list

In [ ]:
#Getting the column headings from soup as a list
presidential_entities = soup.find_all('th')[:7] #pick only the first 7 headers. the rest do not belong in the table

presidential_entities = [entity.text.strip() for entity in presidential_entities]

##### Cleaning column names using regular expressions

In [ ]:
#remove citation superscripts in square brackets
presidential_entities = [re.sub(r'\[.*?\]', '', entity) for entity in presidential_entities] 

#remove Birth-Death string
presidential_entities = [re.sub(r'\(Birth–Death\)', '', entity) for entity in presidential_entities] 

#exclude Portrait table column heading
presidential_entities = [entity for entity in presidential_entities if entity != 'Portrait'] 

presidential_entities

### Creating the dataframe

#### df is the dataframe that will be atomized further to answer the questions
#### df2 is the dataframe that contains the exact table data from wikipedia

In [ ]:
df = pd.DataFrame(columns = presidential_entities) #creating dataframe using presidential entity list as columns

df2 = df.copy() #creating dataframe using presidential entity list as columns

### Populating the dataframe

In [ ]:
column_data = table.find_all('tr') #find table row tags

In [ ]:
#df = pd.DataFrame(columns = presidential_entities)
#column_data = table.find_all('tr') #find table row tags
#for row in column_data[1:]:
#    row_data = row.find_all('td')
#    individual_rows = [data.text.strip() for data in row_data]
#    print(individual_rows)
#.   length = len(df)
#    df.loc[length] = individual_rows

##### Adding rows from the soup to the data frame and cleaning data with regular expresions and string methods using a loop

In [ ]:
for i,row in enumerate(column_data[1:]): #skip the ['1']
    row_data = row.find_all('td')[1:]
    individual_rows = [data.text.strip() for data in row_data]
    individual_rows = [re.sub(r'\[.*?\]', '', row) for row in individual_rows] #remove superscripts on names
    individual_rows = [row for row in individual_rows if row] #list-comprehension syntax to remove empty strings
    individual_rows = [row.replace('\n\n', ' & ') for row in individual_rows] #placing & between multiple election years
    individual_rows = [re.sub(r'– &', '', row) for row in individual_rows] #remove - & from single election years
    individual_rows = [row.replace('\xa0', ' ') for row in individual_rows]  #replacing non-breaking space with regular space
    individual_rows = [row.replace('\n', ' ') for row in individual_rows] #placing & between multiple election years
    individual_rows = [re.sub(r'Vacant after[^\&]*', '', row) for row in individual_rows]  # clean Vacant after from Vice President
    individual_rows = [re.sub(r'Vacant throughout[^\&]*', '', row) for row in individual_rows]  # clean Vacant throughout
    individual_rows = [re.sub(r'Vacant through[^\&]*', '', row) for row in individual_rows]  # clean Vacant through
    individual_rows = [re.sub(r'Vacant:[^&]*', '', cell) for cell in individual_rows]  # Vacant:<date range>
    individual_rows = [re.sub(r' & & ', ' & ', row) for row in individual_rows] #remove & & replace w &
    individual_rows = [re.sub(r'^\s*&\s*', '', row) for row in individual_rows]  # remove & from the start vp name
    individual_rows = [re.sub(r'\s*&\s*$', '', row) for row in individual_rows]  # Remove & from the end of vp names
    individual_rows.insert(0, str(i+1)) #enumerating the rows by inserting their count number in the array
    
    #print(individual_rows)
    
    length = len(df)
    
    df.loc[length] = individual_rows
    df2.loc[length] = individual_rows #or df2 = df.copy() in outer indent

##### Further data cleaning and pre-processing for Question 5 & 11

In [ ]:
#df = df2.copy() #prevents errors when this cell is run multiple times. otherwise comment out

#create new column with Birth and Death Years
birth_death_years = df['Name'].str.extract(r'\((\d{4})–(\d{4})\)')
df[['Birth Year', 'Death Year']] = birth_death_years


#edit NaN values in birthyear with presidents who are alive. Use[0] to extract and pass series to fill na.
birth_years = df['Name'].str.extract(r'\(b\. (\d{4})\)')[0]
df['Birth Year'] = df['Birth Year'].fillna(birth_years)

#remove birth death information from Name series
df['Name'] = df['Name'].str.replace(r'\(\d{4}–\d{4}\)', '', regex=True) #raw string pattern for (1913–2006)
df['Name'] = df['Name'].str.replace(r'\(b\. \d{4}\)', '', regex=True) #raw string pattern for (b. 1942)

df2['Name'] = df2['Name'].str.replace(r'\(\d{4}–\d{4}\)', '', regex=True) #raw string pattern for (1913–2006)
df2['Name'] = df2['Name'].str.replace(r'\(b\. \d{4}\)', '', regex=True) #raw string pattern for (b. 1942)


#extract date in and date out from term column. drop term column
df[['Date In', 'Date Out']] = df['Term'].str.split('–', expand=True)
df.drop('Term', axis = 1, inplace = True)

# convert date columns to datetime using pandas. coerce to handle non date elements
df['Date In'] = pd.to_datetime(df['Date In'], errors='coerce')
df['Date Out'] = pd.to_datetime(df['Date Out'], errors='coerce')

#filling NaN death years with current year
current_date = datetime.now().year
df['Death Year'] = df['Death Year'].fillna(current_date)

# Question 1
Web Scraping

In [ ]:
df2

# Question 2
Number of presidents

In [ ]:
num_presidents = df['Name'].nunique()
print(f"There are {num_presidents} United States of America presidents on the webpage")

# Question 3
Number of presidencies

In [ ]:
num_presidencies = df['No.'].nunique()
print(f"There have been {num_presidencies} preisdencies in the United States of America till date")

# Question 4
Presidents who served more than 1 term with the same vice president

In [ ]:
duration = (df['Date Out'] - df['Date In']).dt.days

for index, row in df.iterrows():
    p = row['Name']
    election = row['Election']
    term = df2['Term'].iloc[index]
    vp = row['Vice President']
    time = duration[index] 
    
    # Check if the president served more than 1 term (duration > 1461 days) and had only one vice president
    if time > 1461 and '&' not in vp:
        print(f"{p} had served more than 1 term: {term} days with one vice president: {vp}\n")

# Question 5
President with the longest term

In [ ]:
service_duration = df['Date Out'].dt.year - df['Date In'].dt.year #create standalone series with service durations
longest_serving_president = df.loc[service_duration.idxmax()] #return index of max duration into loc to return row
print(f"The longest serving president was {longest_serving_president['Name']}, " + 
      f"he served {int((service_duration.iloc[service_duration.idxmax()])/4)} terms " +
      f"and entered office in {df['Date In'].dt.date.iloc[service_duration.idxmax()]}. \n" + 
      f"He was elected to office in {df['Election'].iloc[service_duration.idxmax()]}. "+
      f"If he had not died in 1945, he would have served his 4th term.")

# Question 6
Presidents who served as vice presidents

In [ ]:
vice_presidents = []
# iterate over each row in the Vice President Series
for vp in df['Vice President'].dropna(): #ignore null entries
    # split the vp string by & and append each name to the list
    for name in vp.split('&'):
        vice_presidents.append(name.strip()) #remove trailing and leading whitespaces
    
presidents = []
# iterate over each row in the President Series
for p in df['Name']:
    # split the string by & and append each name to the list
    presidents.append(p.strip()) #remove trailing and leading whitespaces
    
vps_to_presidents = []

for vp in vice_presidents:
    if vp in presidents:
        vps_to_presidents.append(vp)
        
#df2['Term'].iloc[df2.index[df2['Name'] == 'John Adams']].iloc[0] #last iloc[0] is very important
presidency_service = [df2['Term'].iloc[df2.index[df2['Name'] == name]].iloc[0] for name in vps_to_presidents]

#vp_service = [df2['Term'].iloc[df2.index[df2['Vice President'] == name]].iloc[0] for name in vps_to_presidents]
vp_service = []

for name in vps_to_presidents:
    #using boolean indexing for series to filter rows that have the current vp name in the VP column
    #boolean condition
    is_name_in_vp_column = df2['Vice President'].str.contains(name, na=False)
    filtered_rows = df2[is_name_in_vp_column] #boolean series
    
    if not filtered_rows.empty:
        term = filtered_rows['Term'].iloc[0]
        vp_service.append(term)
        
#print(f'Presidents who were vice presidents: {", ".join(vps_to_presidents)}')
print(f'Presidents who were vice presidents.')

for vp, vp_period, p_period in zip(vps_to_presidents,vp_service,presidency_service):
    print(vp + " served as vice from " + vp_period + " and served presidency from " + p_period +".\n")

    
#presidency_service 
#vp_service

#df2['Term'].iloc[df2.index[df2['Vice President'] == 'John Adams']].iloc[0]
#this causes an error hence the need for the longer for loop
#df2['Term'].iloc[df2.index[df2['Vice President'] == 'Gerald Ford']].iloc[0] 

# Question 7
Presidents who were never elected

In [ ]:
count = 0
for index, row in df2.iterrows():
    prez = row['Name']
    term = row['Term']
    if row['Election'] == '–':
        count += 1
        print(f"{prez} was never elected. However, his term was from {term}.\n")
print(f"In total, {count} presidents were never elected.")

# Question 8
Presidents who have the same forenames or surnames

In [ ]:
#create list of all names
full_names = df['Name'].tolist()
#full_names = [name for name in full_names if full_names.count(name) < 2] #remove the duplicate
#full_names = [name[0]+name[-1] for name.split(':') in full_names]
for i, name in enumerate(full_names):
    names_list = name.split(' ')
    full_names[i] = str(i) + ' ' + names_list[0]+ ' ' +names_list[-1]

#create histogram dictionary of forenames
forenames = {} #president name histogram
forenames_list = {}
for prez in full_names:
    forenames[prez.split(' ')[1]] = forenames.get(prez.split(' ')[1],0)+1
    forenames_list[prez.split(' ')[0]] = prez.split(' ')[1]
    #same_first_names
    
#create histogram dictionary of surnames
surnames = {} #president name histogram
surnames_list = {}
for prez in full_names:
    surnames[prez.split(' ')[2]] = surnames.get(prez.split(' ')[2],0)+1
    surnames_list[prez.split(' ')[0]] = prez.split(' ')[2]
    
#create list of same forenames
#same_forename_count = 0
same_forenames = []
same_forenames_idx = []
for i in forenames:
    if forenames[i]>1:
        #same_forename_count += forenames[i]
        same_forenames.append(i)
        #print(i)
        for index in forenames_list:
            if forenames_list[index] == i:
                same_forenames_idx.append(int(index))


#print(same_forenames)   
#print(same_forenames_idx) 
#print('')

#create list of same surnames
#same_surname_count = 0
same_surnames = []
same_surnames_idx = []
for i in surnames:
    if surnames[i]>1:
        #same_surname_count += surnames[i]
        same_surnames.append(i)
        #print(i)
        for index in surnames_list:
            if surnames_list[index] == i:
                same_surnames_idx.append(int(index))
                
#print(same_surnames)  
#print(same_surnames_idx) 

# for i in same_forenames:
#     print(df['Name'].iloc[df.index[df['Name'] == name]].iloc[0] for name in forenames_list)

same_forenames = [df['Name'].iloc[idx] for idx in same_forenames_idx]
same_surnames = [df['Name'].iloc[idx] for idx in same_surnames_idx]

# converting the lists to a set to remove duplicates then back to a list and sorting them afterwards
same_forenames = sorted(list(set(same_forenames)))
same_surnames = sorted(list(set(same_surnames)), key=lambda x: x.split()[-1]) #sort on last name w lambda function

# removing Grover Clevelan from the lists
if 'Grover Cleveland' in same_forenames:
    same_forenames.remove('Grover Cleveland')
if 'Grover Cleveland' in same_surnames:
    same_surnames.remove('Grover Cleveland')
        
print("These are the names of presidents with the same forenames and when they served in office\n")
for i,name in enumerate(same_forenames):
    print(name + ' ||| served from ' + df2['Term'].iloc[same_forenames_idx[i]])
    
print('')

print("These are the names of presidents with the same surnames and when they served in office\n")
for i,name in enumerate(same_surnames):
    print(name + ' ||| served from ' + df2['Term'].iloc[same_surnames_idx[i]])

# Question 9
Oldest president ever elected for office

In [ ]:
year_in = df['Date In'].dt.year
birth_year = pd.to_datetime(df['Birth Year'],format='%Y').dt.year
oldest_index = (year_in - birth_year).idxmax()

#creating a new data frame to verify result of index 45
df_election_age = pd.DataFrame({'Date In': year_in, 'Birth Year': birth_year})
df_election_age['Election Age'] = year_in - birth_year

print(f"{df['Name'].iloc[oldest_index]} is the oldest president ever elected for office, " +
      f"he got in office {df['Date In'].dt.year.iloc[oldest_index]}, " +
      f"he ended {df['Date Out'].dt.date.iloc[oldest_index] if pd.notna(df['Date Out'].iloc[oldest_index]) else 'not yet'} and his tenure lasted " + 
      f"{df['Date Out'].dt.year.iloc[oldest_index] - df['Date In'].dt.year.iloc[oldest_index] if pd.notna(df['Date Out'].iloc[oldest_index]) else (datetime.now().year-year_in.iloc[oldest_index])} years")

# Question 10
Youngest president ever elected for office

In [ ]:
year_in = df['Date In'].dt.year
birth_year = pd.to_datetime(df['Birth Year'],format='%Y').dt.year
youngest_index = (year_in - birth_year).idxmin() #this equals 25

print(f"{df['Name'].iloc[youngest_index]} is the youngest president ever elected for office, " +
      f"he got in office {df['Date In'].dt.year.iloc[youngest_index]}, " +
      f"he ended {int(df['Date Out'].dt.year.iloc[youngest_index])} and his tenure lasted " + 
      f"{int(df['Date Out'].dt.year.iloc[youngest_index]-df['Date In'].dt.year.iloc[youngest_index])} years")

# Question 11
President who lived very long after leaving office

In [ ]:
year_out = df['Date Out'].dt.year
death_year = pd.to_datetime(df['Death Year'],format='%Y').dt.year
longest_living = (death_year - year_out).idxmax()

#creating a new data frame to verify result of index 8
# df_longevity = pd.DataFrame({'Date Out': year_out, 'Death Year': death_year})
# df_longevity['Years Lived After Office'] = death_year - year_out
# df_longevity

print(f"{df['Name'].iloc[longest_living]} is the president who has lived longest after leaving office, " +
      f"he left office on {df['Date Out'].dt.date.iloc[longest_living]}, " +
      f"he was alive for {int((death_year - year_out).iloc[longest_living])} years after leaving office.")

# Question 12
Graph of political party against number of presidents elected

In [ ]:
party_counts = df['Party'].value_counts().reset_index()

party_counts.columns = ['Political Party', 'Number of Presidents']

party_counts.plot.bar(x='Political Party', y='Number of Presidents', rot=0, figsize=(30, 10))

max_party = party_counts['Number of Presidents'].idxmax()

print(f"The party with the most presidents is the {party_counts['Political Party'].iloc[max_party]} party with " + 
      f"{party_counts['Number of Presidents'].iloc[max_party]} presidents elected")

# References

Geeks For Geeks. (2020, July 1). Find maximum values & position in columns and rows of a Dataframe in Pandas. GeeksforGeeks. https://www.geeksforgeeks.org/find-maximum-values-position-in-columns-and-rows-of-a-dataframe-in-pandas/

Mode. (2016, May 23). How to Filter Data with Boolean Indexes in Python | Python Analysis Tutorial - Mode. Mode Resources. https://mode.com/python-tutorial/python-filtering-with-boolean-indexes

Pandas. (n.d.). pandas.DataFrame.groupby — pandas 1.2.4 documentation. Pandas.pydata.org. Retrieved May 30, 2024, from https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

Python. (n.d.). re — Regular expression operations. Python Documentation. Retrieved May 30, 2024, from https://docs.python.org/3/library/re.html#raw-string-notation

Saturn Cloud. (2023, December 19). How to Convert Strings in a Pandas Dataframe to a Date Data Type | Saturn Cloud Blog. Saturncloud.io. https://saturncloud.io/blog/how-to-convert-strings-in-a-pandas-data-frame-to-a-date-data-type/#:~:text=The%20to_datetime()%20function%20is

StackOverflow. (n.d.). How can I iterate over rows in a Pandas DataFrame? Stack Overflow. Retrieved May 30, 2024, from https://stackoverflow.com/questions/16476924/how-can-i-iterate-over-rows-in-a-pandas-dataframe

Vijetha. (2022, November 24). Pandas Extract Year from Datetime. Spark by {Examples}. https://sparkbyexamples.com/pandas/pandas-extract-year-from-datetime/#